In [1]:
import psutil
import numpy as np
from data.rosette_temp import Rosette
import pandas as pd
from multiprocess import Pool, get_context
from itertools import product
import os
import pyvista as pv
import random

# Test #1: single rosette

In [2]:
import pyvista as pv
import numpy as np
from math import pi, sqrt, pow, acos, degrees, radians, sin, cos
# import trame
import math
# import helper
from data import helper_temp as helper
from copy import deepcopy
import miniball
import pymeshfix as mf
from decimal import *

In [2]:
a = 10
c = 30
r0 = 8.41
hp = r0
h0 = hp/4
n_arms = 5
ros = Rosette(a, c, r0, h0, hp, n_arms)

In [68]:
# test with randomly varying sphere size
a = 10
c = 30
f_r0 = random.uniform(0.75, 1.0)
r0 = f_r0 * a
hp = r0
h0 = hp/4
n_arms = 7
print(a, c, r0, h0, hp, n_arms)
print(type(r0))
ros = Rosette(a, c, r0, h0, hp, n_arms)
ros.unify_mesh()
pl = ros.plot(op=0.95)
pl.show()

10 30 9.994653568334764 2.498663392083691 9.994653568334764 7
<class 'float'>


Widget(value='<iframe src="http://localhost:64653/index.html?ui=P_0x3ee32ed20_64&reconnect=auto" class="pyvist…

In [41]:
# create sphere 
sphere = pv.Sphere(radius=r0, center=(0, 0, 0), direction=(0, 0, 1), 
                theta_resolution=30, phi_resolution=20, start_theta=0, 
                end_theta=360, start_phi=0, end_phi=180)
sphere = sphere.triangulate()

In [5]:
# create outer shell to "place" bullets on
r_outer = hp/2 + c - h0 + r0
outer_sphere = pv.Sphere(radius=r_outer, center=(0, 0, 0), direction=(0, 0, 1), 
                theta_resolution=30, phi_resolution=20, start_theta=0, 
                end_theta=360, start_phi=0, end_phi=180)
if n_arms == 2: # line
    outer_shell = pv.Line(pointa=(-r_outer, 0.0, 0.0), 
                        pointb=(r_outer, 0.0, 0.0), resolution=1)
    outer_coords = outer_shell.points
elif n_arms == 4: # tetrahedron
    outer_shell = pv.Tetrahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 6: # octahedron
    outer_shell = pv.Octahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 8: # cube
    # Note: this may not be the optimal solution for n=8, check later
    l  = (2*r_outer)/(3**(1/2))
    outer_shell = pv.Cube(center=(0.0, 0.0, 0.0), x_length=l, 
                        y_length=l, z_length=l)
    outer_coords = outer_shell.points
else: 
    # Modified fibbonaci lattice 
    # Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
    epsilon = 0.33
    goldenRatio = (1 + 5**0.5)/2
    i = np.arange(0, n_arms) 
    theta = 2 *pi * i / goldenRatio
    phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
    x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
    outer_coords = r_outer*(np.column_stack((x, y, z)))

outer_coords = outer_coords # for testing 

In [ ]:
import math
def round_up_to_decimal(number, decimals=0):
    """Rounds a number up to the nearest specified decimal place.

    Args:
        number: The number to round up.
        decimals: The number of decimal places to round to (default is 0).

    Returns:
        The rounded-up number.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimals must be an integer")
    if decimals < 0:
        raise ValueError("decimals must be non-negative")
    
    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

def round_down_to_decimal(number, decimals=0):
    """Rounds a number down to the nearest specified decimal place.

    Args:
        number: The number to round down.
        decimals: The number of decimal places to round to (default is 0).

    Returns:
        The rounded-down number.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimals must be an integer")
    if decimals < 0:
        raise ValueError("decimals must be non-negative")
    
    factor = 10 ** decimals
    return math.floor(number * factor) / factor

pyr_center_rounded = (0.0, 0.0, round_up_to_decimal(hp/2, 2))
cyl_center_rounded = (0.0, 0.0, round_down_to_decimal(c+hp, 2))
print(pyr_center_rounded)   
print(cyl_center_rounded)

In [17]:
# create bullet arm
cyl = pv.Cylinder(center=cyl_center_rounded, direction=(0.0, 0.0, -1.0), 
                radius=a, height=2*c, resolution=6, capping=True)
pyr = pv.Cone(center=pyr_center_rounded, direction=(0.0, 0.0, -1.0), 
            height=hp, radius=a, capping=True, angle=None, resolution=6)
cyl = cyl.triangulate()
pyr = pyr.triangulate()
cyl_pts = cyl.points
cyl_pts[abs(cyl_pts)<1e-10]=0.0 # replace small values with zeros
cyl.points = cyl_pts
pyr_pts = pyr.points
pyr_pts[abs(pyr_pts)<1e-10]=0.0 # replace small values with zeros
pyr.points = pyr_pts
# bullet = cyl.boolean_union(pyr).triangulate()
# pt_dist = np.linalg.norm(bullet.points, axis=1)

In [18]:
bullet = cyl.boolean_union(pyr)

In [ ]:
# plot bullet as wireframe
pl = pv.Plotter()
pl.add_mesh(bullet, color='black', style='wireframe')
pl.show()

In [ ]:
# plot cyl and pyr with pyvista
pl = pv.Plotter()
pl.add_mesh(cyl, color='blue', style='wireframe')
pl.add_mesh(pyr, color='red', style='wireframe')
pl.show()

In [ ]:
# calculate volume
v_ros = ros.volume
print(v_ros)

In [ ]:
# calculate minimally bounding sphere
mbs = ros.calc_mbs()
print(mbs)

In [ ]:
# calculate effective density 
rho_eff = v_ros / mbs['v']
print(rho_eff)

In [ ]:
# visualize mbs
pl = ros.plot()
# create sphere with radius r using pyvista
sphere = pv.Sphere(radius=mbs['r'], center=mbs['c'])
# plot sphere with pyvista
pl.add_mesh(sphere, color='red', opacity=0.5)
pl.show()


# Test #2: test m

# Test #2: multiple rosettes, single cpu

# Test #3: multiple rosettes, multiple cpus